# TOC:
* [Genarete captions for test data](#genarete_captions_for_test_data)
* [Evaluation](#evaluation)

These two sections implement the same as get_scores.py.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
from PIL import Image
import numpy as np
import json

import tensorflow as tf

import configuration
import inference_wrapper
from inference_utils import caption_generator
from inference_utils import vocabulary

## Genarete captions for test data <a id="genarete_captions_for_test_data"></a>

Please run generate_captions.py, instead of using this section in jupyter notebook.

In [2]:
file_name_list = os.listdir("/cis/phd/cxz2081/data/mscoco/captioning/val2014")
file_name_list = ['/cis/phd/cxz2081/data/mscoco/captioning/val2014/' + i for i in file_name_list]
file_name_list = file_name_list[:5]  # DEBUG
print(file_name_list)

['/cis/phd/cxz2081/data/mscoco/captioning/val2014/COCO_val2014_000000126229.jpg', '/cis/phd/cxz2081/data/mscoco/captioning/val2014/COCO_val2014_000000579325.jpg', '/cis/phd/cxz2081/data/mscoco/captioning/val2014/COCO_val2014_000000499989.jpg', '/cis/phd/cxz2081/data/mscoco/captioning/val2014/COCO_val2014_000000065024.jpg', '/cis/phd/cxz2081/data/mscoco/captioning/val2014/COCO_val2014_000000575897.jpg']


In [3]:
FLAGS = tf.flags.FLAGS

tf.flags.DEFINE_string("checkpoint_path", "model/train", "Model checkpoint file or directory containing a model checkpoint file.")
tf.flags.DEFINE_string("vocab_file", "data/mscoco/word_counts.txt", "Text file containing the vocabulary.")
tf.flags.DEFINE_string("input_files", file_name_list, "File pattern or comma-separated list of file patterns of image files.")
tf.flags.DEFINE_string("captions_file", "/cis/phd/cxz2081/data/mscoco/captioning/annotations/captions_val2014.json", "JSON file containing caption annotations..")


In [17]:
# Extract the filenames and ids.
with tf.gfile.FastGFile(FLAGS.captions_file, "r") as f:
    caption_data = json.load(f)

filename_to_id = {"/cis/phd/cxz2081/data/mscoco/captioning/val2014/" + x["file_name"]: x["id"] for x in caption_data["images"]}
id_to_filename = {x["id"]: "/cis/phd/cxz2081/data/mscoco/captioning/val2014/" + x["file_name"] for x in caption_data["images"]}


In [4]:
# Build the inference graph.
g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(configuration.ModelConfig(),
                                           FLAGS.checkpoint_path)
g.finalize()

# Create the vocabulary.
vocab = vocabulary.Vocabulary(FLAGS.vocab_file)

filenames = []
for file_pattern in FLAGS.input_files:
    filenames.extend(tf.gfile.Glob(file_pattern))
tf.logging.info("Running caption generation on %d files matching.",
              len(filenames))

INFO:tensorflow:Building model.
INFO:tensorflow:Initializing vocabulary from file: data/mscoco/word_counts.txt
INFO:tensorflow:Created vocabulary with 11520 words
INFO:tensorflow:Running caption generation on 5 files matching.


In [6]:
# generate captions
with tf.Session(graph=g) as sess:
    # Load the model from checkpoint.
    restore_fn(sess)

    # Prepare the caption generator. Here we are implicitly using the default
    # beam search parameters. See caption_generator.py for a description of the
    # available beam search parameters.
    generator = caption_generator.CaptionGenerator(model, vocab)

    captions_results = []
    for filename in filenames:
        with tf.gfile.GFile(filename, "r") as f:
            image = f.read()
        captions = generator.beam_search(sess, image)
        print("Captions for image %s:" % os.path.basename(filename))
        for i, caption in enumerate(captions):
            # Ignore begin and end words.
            sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
            sentence = " ".join(sentence)
            if i == 0:
                # save the caption with highest score.
                captions_results.append({'image_id': filename_to_id[filename], 'caption':sentence})
            print("  %d) %s (p=%f)" % (i, sentence, math.exp(caption.logprob)))

INFO:tensorflow:Loading model from checkpoint: model/train/model.ckpt-829178
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-829178
Captions for image COCO_val2014_000000126229.jpg:
  0) a tennis player on a court with a racket (p=0.001882)
  1) a tennis player on the court with a racket (p=0.000167)
  2) a tennis player on a court with a tennis racket (p=0.000117)
Captions for image COCO_val2014_000000579325.jpg:
  0) a large jetliner sitting on top of an airport tarmac . (p=0.010742)
  1) a large jetliner sitting on top of an airport runway . (p=0.007201)
  2) a large passenger jet sitting on top of an airport tarmac . (p=0.004929)
Captions for image COCO_val2014_000000499989.jpg:
  0) a giraffe standing in the middle of a field . (p=0.003387)
  1) a giraffe standing in the middle of a forest . (p=0.001570)
  2) a giraffe standing in the middle of a lush green field . (p=0.001259)
Captions for image COCO_val2014_000000065024.jpg:
  0) a panda bear sitting on top of a log .

In [18]:
# # save captions in json format, as a list of dict
# with open('results/mscoco_captioning_results.json', 'w') as f:
#     json.dump(captions_results,f)

'''
NOTE: Saving json file to disk is problemetic in jupyuter notebook. 
use python file to save it as json.
'''

'\nNOTE: Saving json file to disk is problemetic in jupyuter notebook. \nuse python file to save it as json.\n'

## Evaluation <a id="evaluation"></a>

In [8]:
%matplotlib inline
from coco_caption.pycocotools.coco import COCO
from coco_caption.pycocoevalcap.eval import COCOEvalCap
import matplotlib.pyplot as plt
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (5.0, 4.0)

import json
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

In [1]:
# set up file names and pathes
homeDir = '/cis/phd/cxz2081'
dataSetDir = 'data/mscoco/captioning'
dataDir='.'
annFile='%s/%s/annotations/captions_val2014_addtype.json'%(homeDir,dataSetDir)
subtypes=['results', 'evalImgs', 'eval']
[resFile, evalImgsFile, evalFile]= \
['%s/results/captions_selftest2014_rhn_%s.json'%(dataDir,subtype) for subtype in subtypes]

print(annFile)
print(resFile)
print(evalImgsFile)
print(evalFile)

/cis/phd/cxz2081/data/mscoco/captioning/annotations/captions_val2014_addtype.json
./results/captions_selftest2014_rhn_results.json
./results/captions_selftest2014_rhn_evalImgs.json
./results/captions_selftest2014_rhn_eval.json


In [23]:
# create coco object and cocoRes object
coco = COCO(annFile)
cocoRes = coco.loadRes(resFile)

loading annotations into memory...
0:00:01.035807
creating index...
index created!
Loading and preparing results...     
DONE (t=0.03s)
creating index...
index created!


In [30]:
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate on a subset of images by setting
# cocoEval.params['image_id'] = cocoRes.getImgIds()
# please remove this line when evaluating the full validation set
cocoEval.params['image_id'] = cocoRes.getImgIds()

# evaluate results
cocoEval.evaluate()

tokenization...
setting up scorers...
computing Bleu score...
{'reflen': 45, 'guess': [48, 43, 38, 33], 'testlen': 48, 'correct': [38, 20, 9, 3]}
ratio: 1.06666666664
Bleu_1: 0.792
Bleu_2: 0.607
Bleu_3: 0.443
Bleu_4: 0.298
computing METEOR score...
METEOR: 0.243
computing Rouge score...
ROUGE_L: 0.548
computing CIDEr score...
CIDEr: 1.194


In [27]:
# print output evaluation scores
for metric, score in cocoEval.eval.items():
    print('%s: %.3f'%(metric, score))

CIDEr: 1.194
Bleu_4: 0.298
Bleu_3: 0.443
Bleu_2: 0.607
Bleu_1: 0.792
ROUGE_L: 0.548
METEOR: 0.243


In [ ]:
# # demo how to use evalImgs to retrieve low score result
# evals = [eva for eva in cocoEval.evalImgs if eva['CIDEr']<30]
# print 'ground truth captions'
# imgId = evals[0]['image_id']
# annIds = coco.getAnnIds(imgIds=imgId)
# anns = coco.loadAnns(annIds)
# coco.showAnns(anns)

# print '\n'
# print 'generated caption (CIDEr score %0.1f)'%(evals[0]['CIDEr'])
# annIds = cocoRes.getAnnIds(imgIds=imgId)
# anns = cocoRes.loadAnns(annIds)
# coco.showAnns(anns)

# img = coco.loadImgs(imgId)[0]
# I = io.imread('%s/images/%s/%s'%(dataDir,dataType,img['file_name']))
# plt.imshow(I)
# plt.axis('off')
# plt.show()

In [ ]:
# # plot score histogram
# ciderScores = [eva['CIDEr'] for eva in cocoEval.evalImgs]
# plt.hist(ciderScores)
# plt.title('Histogram of CIDEr Scores', fontsize=20)
# plt.xlabel('CIDEr score', fontsize=20)
# plt.ylabel('result counts', fontsize=20)
# plt.show()

In [28]:
# save evaluation results to ./results folder
json.dump(cocoEval.evalImgs, open(evalImgsFile, 'w'))
json.dump(cocoEval.eval,     open(evalFile, 'w'))